In [21]:
import pandas as pan;
import numpy as np;
import matplotlib.pyplot as plotter;
import seaborn as sns;
from sklearn.metrics import mean_absolute_error, mean_squared_error;
from sklearn.model_selection import GridSearchCV, train_test_split;
from sklearn.preprocessing import PolynomialFeatures, StandardScaler;
from sklearn.linear_model import ElasticNetCV, ElasticNet;
from joblib import load, dump


In [7]:
matrix_data = pan.read_csv("OneHotEncodedCleanData.csv")
matrix_data.columns

Index(['MS SubClass_150', 'MS SubClass_160', 'MS SubClass_180',
       'MS SubClass_190', 'MS SubClass_20', 'MS SubClass_30', 'MS SubClass_40',
       'MS SubClass_45', 'MS SubClass_50', 'MS SubClass_60',
       ...
       'Wood Deck SF', 'Open Porch SF', 'Enclosed Porch', '3Ssn Porch',
       'Screen Porch', 'Pool Area', 'Misc Val', 'Mo Sold', 'Yr Sold',
       'SalePrice'],
      dtype='object', length=263)

In [8]:
label = matrix_data["SalePrice"]
features = matrix_data.drop("SalePrice", axis = 1)
label_2 = label
features_2 = features

In [9]:
#lets see whether a polynomial features algorithm with ElasticNet will do better
poly_features2_maker = PolynomialFeatures(degree = 2, include_bias = False)
poly_features2 = poly_features2_maker.fit_transform(features_2)
#perform train-test-split for the 2 of them
feature_train, feature_test, label_train, label_test = train_test_split(features, label, random_state = 101, test_size = 0.1)
feature_2train, feature2_test, label_2train, label2_test = train_test_split(poly_features2, label, random_state= 101, test_size = 0.1)
#initialize 2 scalers for the data
scaler = StandardScaler()
scaler_2 = StandardScaler()
scaler.fit(feature_train)
scaler_2.fit(feature_2train)
feature_train = scaler.transform(feature_train)
feature_test = scaler.transform(feature_test)
feature2_test = scaler_2.transform(feature2_test)
feature_2train = scaler_2.transform(feature_2train)

In [10]:
#models
elastic_learner = ElasticNetCV(l1_ratio = [0.05, 0.1, 0.2, 0.5, 0.7, 0.9, 0.95, 1], n_alphas = 100, eps = 0.1, max_iter = 100000, n_jobs = 3)
sub_gridder = ElasticNet()

In [11]:
#elastic_learner.fit(feature_2train, label_2train)
params = {"alpha": [0.1, 1, 10, 50, 100], "l1_ratio": [0.1, 0.5, 0.7, 0.9, 0.95, 1]}
gridder = GridSearchCV(sub_gridder, params, scoring = ["neg_mean_absolute_error", "neg_mean_squared_error"], n_jobs = 3, refit = "neg_mean_squared_error", verbose = 3)

In [12]:
gridder.fit(feature_train, label_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


GridSearchCV(estimator=ElasticNet(), n_jobs=3,
             param_grid={'alpha': [0.1, 1, 10, 50, 100],
                         'l1_ratio': [0.1, 0.5, 0.7, 0.9, 0.95, 1]},
             refit='neg_mean_squared_error',
             scoring=['neg_mean_absolute_error', 'neg_mean_squared_error'],
             verbose=3)

In [13]:
results = pan.DataFrame(gridder.cv_results_)

In [15]:
gridder.best_estimator_

ElasticNet(alpha=100, l1_ratio=1)

In [17]:
results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_l1_ratio,params,split0_test_neg_mean_absolute_error,split1_test_neg_mean_absolute_error,split2_test_neg_mean_absolute_error,...,std_test_neg_mean_absolute_error,rank_test_neg_mean_absolute_error,split0_test_neg_mean_squared_error,split1_test_neg_mean_squared_error,split2_test_neg_mean_squared_error,split3_test_neg_mean_squared_error,split4_test_neg_mean_squared_error,mean_test_neg_mean_squared_error,std_test_neg_mean_squared_error,rank_test_neg_mean_squared_error
0,0.301631,0.028131,0.002194,9.752278e-04,0.1,0.1,"{'alpha': 0.1, 'l1_ratio': 0.1}",-16528.768432,-15751.270513,-13924.853196,...,969.175568,9,-6.960951e+08,-6.308176e+08,-3.807568e+08,-4.124125e+08,-5.925300e+08,-5.425224e+08,1.240791e+08,11
1,0.266988,0.024882,0.002395,1.352220e-03,0.1,0.5,"{'alpha': 0.1, 'l1_ratio': 0.5}",-16373.489125,-15739.287832,-13911.916871,...,937.421361,6,-6.830834e+08,-6.073710e+08,-3.794546e+08,-4.084604e+08,-5.874240e+08,-5.331587e+08,1.184097e+08,8
2,0.258413,0.013615,0.002790,1.466847e-03,0.1,0.7,"{'alpha': 0.1, 'l1_ratio': 0.7}",-16276.926907,-15747.551209,-13938.791144,...,910.880231,3,-6.753977e+08,-5.959368e+08,-3.794630e+08,-4.078062e+08,-5.863711e+08,-5.289950e+08,1.151117e+08,6
3,0.558892,0.047568,0.002297,5.960762e-04,0.1,0.9,"{'alpha': 0.1, 'l1_ratio': 0.9}",-16169.294236,-15788.130461,-13979.100181,...,879.275042,5,-6.672295e+08,-5.878745e+08,-3.808922e+08,-4.113033e+08,-5.881483e+08,-5.270895e+08,1.112136e+08,3
4,1.054124,0.060410,0.002195,4.016432e-04,0.1,0.95,"{'alpha': 0.1, 'l1_ratio': 0.95}",-16161.498446,-15811.956569,-14001.430400,...,871.564989,8,-6.662464e+08,-5.885859e+08,-3.815021e+08,-4.143817e+08,-5.898214e+08,-5.281075e+08,1.104312e+08,4
5,1.161293,0.051134,0.001993,1.091057e-03,0.1,1,"{'alpha': 0.1, 'l1_ratio': 1}",-16254.969156,-15875.420599,-13980.026118,...,904.381361,12,-6.740347e+08,-6.010229e+08,-3.800511e+08,-4.206663e+08,-5.963385e+08,-5.344227e+08,1.136061e+08,10
6,0.126372,0.029656,0.001398,4.893771e-04,1,0.1,"{'alpha': 1, 'l1_ratio': 0.1}",-17751.899736,-17059.694062,-15250.812090,...,990.567355,16,-8.870600e+08,-9.801418e+08,-4.666645e+08,-5.229363e+08,-7.676137e+08,-7.248833e+08,2.003714e+08,16
7,0.169944,0.011400,0.002010,1.575205e-03,1,0.5,"{'alpha': 1, 'l1_ratio': 0.5}",-17222.752736,-16281.401665,-14498.858822,...,1026.087165,15,-7.977060e+08,-8.281465e+08,-4.159880e+08,-4.675827e+08,-6.791358e+08,-6.377118e+08,1.683359e+08,14
8,0.153589,0.015398,0.002394,1.018291e-03,1,0.7,"{'alpha': 1, 'l1_ratio': 0.7}",-16952.002492,-15950.630657,-14147.777790,...,1036.007079,13,-7.504577e+08,-7.397258e+08,-3.952848e+08,-4.398586e+08,-6.345887e+08,-5.919831e+08,1.487208e+08,13
9,0.273288,0.034115,0.003391,1.352114e-03,1,0.9,"{'alpha': 1, 'l1_ratio': 0.9}",-16557.646528,-15752.605108,-13923.992182,...,977.278434,10,-6.989340e+08,-6.365898e+08,-3.811269e+08,-4.134608e+08,-5.940010e+08,-5.448225e+08,1.254131e+08,12


In [19]:
pred = gridder.predict(feature_test)

In [20]:
print("MAE: {0}".format(mean_absolute_error(label_test, pred)))
print("RMSE: {0}".format(np.sqrt(mean_squared_error(label_test, pred))))

MAE: 14218.227838430787
RMSE: 20619.3448612665


In [22]:
#train another instance on the given model params
graduate_learner = ElasticNet(alpha = 100, l1_ratio = 1)#could just write a lasso and be done with this, but I'm really partial to this one
finalscaler = StandardScaler()
graduate_learner.fit(finalscaler.fit_transform(features), label)

ElasticNet(alpha=100, l1_ratio=1)

In [23]:
#I unleash you into the world!
dump(graduate_learner, "primusopus.joblib")

['primusopus.joblib']